This notebook is where Elizabeth is developing the code for the decision tree.

`hopkins-knowledge.csv` contains the knowledge base from [here](https://github.com/drdevinhopkins/20_Questions/blob/master/knowledge_base.csv); used only for development.

In [1]:
import pandas as pd
import numpy as np

In [2]:
kn = pd.read_csv('hopkins-knowledge.csv')

y = kn['Animal']
X = kn.loc[:, 'Hair':'Invertebrate']

print('There are {0} objects and {1} features for each object.'.format(y.shape[0], X.shape[1]))

There are 100 objects and 28 features for each object.


In [3]:
kn[kn['Hair'] == 0]

,Animal,Hair,Feathers,Eggs,Milk,Airborne,Aquatic,Predator,Toothed,Backbone,...,Tail,Domestic,Catsize,Mammal,Bird,Reptile,Fish,Amphibian,Insect,Invertebrate
2,bass,0,0,1,0,0,1,1,1,1,...,1,0,0,0,0,0,1,0,0,0
7,carp,0,0,1,0,0,1,0,1,1,...,1,1,0,0,0,0,1,0,0,0
8,catfish,0,0,1,0,0,1,1,1,1,...,1,0,0,0,0,0,1,0,0,0
11,chicken,0,1,1,0,1,0,0,0,1,...,1,1,0,0,1,0,0,0,0,0
12,chub,0,0,1,0,0,1,1,1,1,...,1,0,0,0,0,0,1,0,0,0
13,clam,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
14,crab,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
15,crayfish,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
16,crow,0,1,1,0,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,0
18,dogfish,0,0,1,0,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,0


In [4]:
def dist_from_1(feat_col):
    """
    Returns the absolute distance from 1 of the split cardinality ratio for the given column of X.
    """
    counts = feat_col.value_counts()
    if len(counts) == 2:  # i.e. if there are both 1s and 0s in the column
        ratio = counts[0] / counts[1] 
        return abs( 1 - ratio )
    return 10e10  # some arbitrarily large value; not the minimum


def feat_nearest_1(df):
    """
    Returns the feature (column name) of the passed-in dataframe with the split cardinality ratio nearest 1,
    as well as the value of that ratio.
    """
    return df.apply(dist_from_1).idxmin(), df.apply(dist_from_1).min()

feat_to_split_on, dist_from_one = feat_nearest_1(X)
feat_to_split_on
# Probably also want a feat_nearest_3 or something, a top 3 kind of thing, in case we have to choose another question 
# if the answer is 'unknown'


def rank_features(df):
    """
    Ranks all features in df by increasing absolute distance from 1 of the SCR.
    """
    return df.apply(dist_from_1).sort_values()

rank_features(X).index[0]  # returns index of 0th element, i.e. feature name
rank_features(X)[0]        # returns value of 0th element

0.18181818181818177

In [5]:
# print(feat_to_split_on, '?')
# answ = int( input() )
# X[X[feat_to_split_on] == answ]

# Proof of concept

In [6]:
def play(X, y, answers):
    
    feat_to_split_on, dist_from_one = feat_nearest_1(X)
    
    # As long as we have more than one row and more than one column, and
    # 10e10 is the flagged value, meaning that there is no more sensible split that can be made between features because
    # they all contain either all 0s or all 1s
        
    if len(X) > 1 and len(X.columns) > 1 and dist_from_one != 1000:

        print(feat_to_split_on, '?')
        answ = int( input() )
        
        answers[feat_to_split_on] = answ
        
        # Prune X to contain only those instances corresponding to the answer, and drop the feature
        X_new = X[X[feat_to_split_on] == answ].drop(columns=[feat_to_split_on])
        
#         print(X_new.index)
        
        play(X_new, y, answers)
    
    else:
        
        if len(X) == 1:
            guess = y[X.index]
        else:
            rd_guess_idx = np.random.choice(X.index)
            guess = y[rd_guess_idx]

        print('\n\nGUESS:', guess)
        print('ANSWERS:', answers)
        
#         return guess, answers  # problems with this for some reason

In [7]:
# play(X, y, dict())

In [8]:
# for gorilla, somehow the whole pandas series is printed out, while when the choice is random, this doesn't happen. why?? 
# Series.to_string(index=False)
# play(X, y, dict())

# Non-deterministic

- DONE random choice of feature, if multiple ones have the same 1-SCR
- if multiple objects left at the end that cannot be distinguished, cycles through all of them randomly and asks in each case until one is guessed
- still only yes/no answers
- add question counter

In [9]:
def play2(X, y, answers):
    """
    Recursively splits knowledge base based on user input about whether target object matches the feature.
    """
    
    # Get list of features, ranked by distance of their SCR from 1 (i.e. sorted by increasing 1-SCR), and identify min.
    ranked_feats = rank_features(X)
    min_feat_diffc = ranked_feats[0]
    
    # Recursive case: As long as X as more than one column and more than one row and the min is not 1000 (= the flag
    # value for when the feature contains only 0s or only 1s).    
    if len(X) > 1 and min_feat_diffc != 10e10: # and len(X.columns) > 1 

        # Check whether multiple features share the same min_feat_diffc.
        dupd_minimum = np.in1d(ranked_feats, min_feat_diffc).sum() != 1

        # If they do, identify all of the features with that min_feat_diffc and randomly choose between them.
        if dupd_minimum:
            min_feats = ranked_feats.index[ np.where(ranked_feats == min_feat_diffc) ]
            print('  Min feats:', min_feats)
            
            feat_to_split_on = np.random.choice(min_feats)
        else:
            feat_to_split_on = ranked_feats.index[0]
        
        print('\n', feat_to_split_on, '?')  # BUILD IN ANNA'S FEATURE NAME -> NATURAL LANGUAGE QUESTION FUNCTION HERE
        answ = int( input() )
        
        answers[feat_to_split_on] = answ
        
        # Prune X to contain only those instances corresponding to the answer, and drop the feature
        X_new = X[X[feat_to_split_on] == answ].drop(columns=[feat_to_split_on])
        
        play2(X_new, y, answers)
    
    # Base case.
    else:
        
        # If there is only one row left in the data, only only one object available to guess.
        if len(X) == 1:
            guess = y[X.index]
            
        # If there are many candidates, cycle through all of them and ask about each one.
        else:
            
            # Jupyter notebook keeps getting hung up when in this loop if I execute this cell multiple times.
            # Ah. I wonder if it's because I'm doing in-place changes to things that I shouldn't be changing, so it gets
            # stuck in infinite loops.
            rem_idcs = list(X.index)
            print('\nRemaining indices:', rem_idcs)
            rd_guess_idx = np.random.choice(rem_idcs)
            guess = y[rd_guess_idx]

        print('\n\nGUESS:', guess)
#         print('ANSWERS:', answers)
        
        print('Correct?')
        guessed_right = int( input() )
        
        msg = 'oh yeah! we rock' if guessed_right == 1 else 'dangit'
        print(msg)
    
# play2(X[['Hair', 'Feathers']], y, dict())

In [10]:
print( y[y.index.isin([24, 30])] )

24    flea
30    goat
Name: Animal, dtype: object


# Attempt 2, flipping base and recursive case

In [20]:
def ask_about_feature(feat_name):
    """
    ANNA: Modify this function to print out a natural language question based on the feature name,
    e.g. "Does it have wings?"
    
    Arg:
        feat_name: string, name of feature to split dataset on
    Prints:
        A string, the natural language question asking about that feature.
    Returns:
        Nothing.
    """
    print(feat_name+'?')


def ask_about_object(obj_name):
    """
    ANNA: Modify this function to print out a natural language question based on the object name, 
    e.g. "Are you thinking of an ocelot?"
    
    Arg:
        obj_name: string, name of object to guess.
    Prints:
        A string, the natural language question guessing that object.
    Returns:
        Nothing.
    """
    print(obj_name+'?')
    
    
def split_df_on_feature(df, feature, answer):
    """
    Returns subset of df where df[feature]==answer and drops feature from columns in df.
    
    Args:
        df: pandas dataframe with features as columns, populated by 0s and 1s, one row per instance
        feature: string, the column name to split on
        answer: int, 0 or 1, reflecting which subset of the dataframe to keep
    Returns:
        pandas dataframe with features as columns (subset of df).
    """
    return df[df[feature] == answer].drop(columns=[feature])
    
# split_df_on_feature(X, 'Hair', 1)

def endgame_lose():
    """
    RODRIGO: If the game is lost, we will need to figure out why (was the 20Q limit reached? Or was the user's 
    object not in the knowledge base?) and take action based on that. The code to add in unknown objects could
    be incorporated here.
    """
    print('dangit')

        
def endgame_win():
    """
    RODRIGO: Does something in the event that the game was won.
    """
    print('oh yeah! I rock')
    

def ask_and_split_on_answer(feature, df):
    """
    Prints question about the supplied feature, gets the answer, and then splits the dataset, returning
    only those instances where the answer holds.
    
    Args:
        feature: a string, a column in df
        df: pandas dataframe with features as columns, populated by 0s and 1s, one row per instance
    Returns:
        pandas dataframe with features as columns (subset of df).
    """
    ask_about_feature(feature)
    answ = int( input() )
    return split_df_on_feature(df, feature, answ)
    
    

In [25]:
def guess_rem_objs(y, X_idcs):
    """
    If dataset cannot be split by features anymore, but multiple objects still remain, this function goes through
    them in a random order, guessing each in turn until endgame.
    
    *** TODO: add counter ***
    
    Args:
        y: pandas series, all objects in dataset
        X_idcs: 'pandas.core.indexes.numeric.Int64Index', the remaining indices to choose from (all non-candidates 
                having been pruned)
    """
    # Subset the ys based on X_idcs and shuffle them, so the guessing will happen in a random order.
    ys_to_guess = y[y.index.isin(X_idcs)]
    ys_to_guess = ys_to_guess.sample(frac=1, random_state=3)

    # Go through ys_to_guess and ask about each object. If guessed correctly, enter endgame_win and stop looping.
    for guess in ys_to_guess:
        ask_about_object(guess)
        answ = int( input() )
        if answ == 1:
            endgame_win()
            break
    
    # If we made it out of the for loop and the final answer isn't 1, that means we lost.
    if answ != 1:
        endgame_lose()

    
def play3(X, y, answers):
    """
    Recursively splits knowledge base based on user input about whether target object matches the feature.
    
    *** TODO: add counter ***
    
    """
    # BASE CASE 0: counter >= 20
    
    # BASE CASE 1: Only one row left in the data, so only one object available to guess.
    if len(X) == 1:
        print('ONLY ONE OBJECT LEFT!')
        guess = y[X.index].to_string(index=False)  # (all this machinery required to print pd.Series as str, sigh)
        ask_about_object(guess)
        answ = int( input() )
        endgame_win() if answ == 1 else endgame_lose()
        return
    
    # BASE CASE 2: Only one feature left in the data (have asked about all other ones). Will need to ask about that feature, 
    # subset the data correspondingly, and then cycle through all remaining objects until endgame.
    if len(X.columns) == 1:
        print('ONLY ONE FEATURE LEFT!')
        feature_to_split_on = X.columns[0]
        X_bc2 = ask_and_split_on_answer(feature_to_split_on, X)
        guess_rem_objs(y, X_bc2.index)  # includes endgame
        return
    
    # BASE CASE 3: All features have either all 0s or all 1s for all remaining objects (observable from the value of
    # min_feat_diffc, which is set to the flag value 10e10 if corresponding feature contains all 0s or all 1s). 
    # Can't divide dataset any more, so will just need to cycle through all remaining objects until endgame.
    
    # Get list of features, ranked by distance of their SCR from 1 (i.e. sorted by increasing 1-SCR), and identify min.
    ranked_feats = rank_features(X)
    min_feat_diffc = ranked_feats[0]
    
    if min_feat_diffc == 10e10:
        print('NO MORE DISTINGUISHING FEATURES!')
        guess_rem_objs(y, X.index)  # includes endgame
        return
    
    # RECURSIVE CASE.
    # If we get this far, that means we didn't fall into any of the base cases, so the game can be played!
    
    # For a lil sprinkling of nondeterminism, we want to randomly choose between multiple features, if they are equally
    # good at splitting the dataset, i.e. if multiple features share the same min_feat_diffc.
    dupd_minimum = np.in1d(ranked_feats, min_feat_diffc).sum() != 1
    if dupd_minimum:
        min_feats = ranked_feats.index[ np.where(ranked_feats == min_feat_diffc) ]
        print('COMPETING FEATURES!', min_feats)
        feat_to_split_on = np.random.choice(min_feats)
    else:
        feat_to_split_on = ranked_feats.index[0]
    
    print(ranked_feats, '\n')  # Sanity check, can rm this later
    X_new = ask_and_split_on_answer(feat_to_split_on, X)
    
    play3(X_new, y, answers)
    
play3(X[['Aquatic', 'Nlegs_5']], y, dict())

Aquatic     0.857143
Nlegs_5    98.000000
dtype: float64 

Aquatic?
1
ONLY ONE FEATURE LEFT!
Nlegs_5?
1
starfish?
0
dangit


In [37]:
X.iloc[[5]]

,Hair,Feathers,Eggs,Milk,Airborne,Aquatic,Predator,Toothed,Backbone,Breathes,...,Tail,Domestic,Catsize,Mammal,Bird,Reptile,Fish,Amphibian,Insect,Invertebrate
5,1,0,0,1,0,0,0,1,1,1,...,1,0,1,1,0,0,0,0,0,0


In [65]:
rank_features( X[['Aquatic', 'Nlegs_5']] )

Aquatic     0.857143
Nlegs_5    98.000000
dtype: float64

In [64]:
kn[kn['Animal'] == 'starfish']

,Animal,Hair,Feathers,Eggs,Milk,Airborne,Aquatic,Predator,Toothed,Backbone,...,Tail,Domestic,Catsize,Mammal,Bird,Reptile,Fish,Amphibian,Insect,Invertebrate
84,starfish,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
